In [1]:
import rasterio 
import matplotlib.pyplot as plt
import numpy as np
import fetch_data as fd 
import geemap 
import ee

In [6]:
# Authenticate and initialize Earth Engine  
fd.authenticate_gee()

# Define the region of interest (ROI)
roi = ee.Geometry.Polygon([
    [[73.3933018836958, 18.88730081255972],
    [73.3933018836958, 18.265071030538323],
    [74.1733311805708, 18.265071030538323],
    [74.1733311805708, 18.88730081255972]]
])

# Load datasets
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

# Define date filters
dateFilter_2019 = ee.Filter.date('2019-01-01', '2019-12-30')
dateFilter_2024 = ee.Filter.date('2024-01-01', '2024-12-15')

# Define metadata filters
cloudFilter = ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 2)

# Cloud Score+ QA bands
qa_band = 'cs_cdf'
clear_threshold = 0.60

# Get filtered images for 2019 and 2024
imgCollection_2019 = fd.filter_image_collection(s2, csPlus, dateFilter_2019, roi, cloudFilter, qa_band, clear_threshold)
imgCollection_2024 = fd.filter_image_collection(s2, csPlus, dateFilter_2024, roi, cloudFilter, qa_band, clear_threshold)

# Create composite images
imgComp2019 = fd.create_image_composite(imgCollection_2019)
imgComp2024 = fd.create_image_composite(imgCollection_2024)

# Clamp images to remove outliers
clampedImage2019 = fd.clamp_img(imgComp2019, roi)
clampedImage2024 = fd.clamp_img(imgComp2024, roi)

# Resample images to 10m resolution
band_list = ['B1', 'B5', 'B6', 'B7', 'B8A', 'B9', 'B11', 'B12']
resampledImage2019 = fd.resample_image_to_10m(clampedImage2019, band_list)
resampledImage2024 = fd.resample_image_to_10m(clampedImage2024, band_list)

In [7]:
# Visualization Parameters
vizParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 88,
    'max': 2443
}

vizParamsScaled = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 1
}

# Create a Map
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add layers
Map.addLayer(resampledImage2019, vizParams, 'Preprocessed 2019')
Map.addLayer(resampledImage2024, vizParams, 'Preprocessed 2024')
Map

Map(center=[18.576397493878297, 73.78331653213331], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
Map2 = geemap.Map()
Map2.centerObject(roi, 10)
image = 'resampledImage2019.tif'
Map2.addLayer(image)
Map2 


# # Open the Sentinel-2 raster file
# with rasterio.open('resampledImage2019.tif') as src:
#     # Read the red, green, and blue bands (assuming bands 4, 3, and 2 are in order)
#     red = src.read(4)
#     green = src.read(3)
#     blue = src.read(2)

# # Normalize the bands for visualization
# red = red / red.max()
# green = green / green.max()
# blue = blue / blue.max()

# # Stack the bands into an RGB image
# rgb = np.dstack((red, green, blue))

# # Plot the RGB image
# plt.figure(figsize=(10, 10))
# plt.imshow(rgb)
# plt.axis('off')
# plt.title('Sentinel-2 RGB Visualization')
# plt.show()

Map(center=[18.576397493878297, 73.78331653213331], controls=(WidgetControl(options=['position', 'transparent_…